<a href="https://colab.research.google.com/github/alicyahyun/alicyahyun.github.io/blob/main/certain_keyword_pos_neg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

트레이닝/테트 세트 분리하기.

A라는 고등학교, B라는 고등학교에서 시험을 치룬다고 했을때,
- A 라는 고등학교는 시험을 보기전에 200문제를 연습문제를 주고 여기서 20문제를 냈고
- B 라는 고등학교는 시험을 보기전에 180문제를 연습문제를 주고 여기서 20문제를 냈다.

B라는 학교처럼 90-80% 학습용 데이터, 10-20% 검즘용 데이터


In [ ]:
import pandas as pd
import numpy as np


df = pd.read_csv("odewratings.csv")
df[:20]

,score,text
0,1,영화내내 가족에게 잘해야겠다 생각이 들었다.우리 아버지는 날위해 모든것을바치신사람이...
1,1,이분들이 있었기에 지금의 내가있고 대한민국이 있는거지. 존경스럽다
2,1,"광부, 간호원 여러분! 나라가 부족하고 내가 부족해 여러분이 여기 있습니다 가족과 ..."
3,1,추운날 한파를 뚫고 시사회로 보고왔으나 돈을 주어도 아깝지 않은 소중한 시간을 선물...
4,1,관람객20대초반 젊은이로 우리가 겪지못했던 우리한아버지들의삶을 보여주는 영화..그들...
5,1,관람객아 눈물범벅. 고생하신 아버지 생각나네요 ㅠㅠ
6,1,관람객조조로 보고 왔습니다. 시작하고 계속 눈물 닦다가 웃다가 했습니다. 근래에 본...
7,1,내일을 조국과 민족을 위하여.
8,1,시사회때 봤는데 단연코 수작임. 2014년 대표영화로 손꼽을만함.근데 방금개봉했는데...
9,1,관람객황정민이캐릭터잘살렷고웃기면서 재밋엇어요


In [ ]:
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

In [ ]:
text = df['text']
score = df['score']

In [ ]:
print(len(df))

38335


In [ ]:
print(df.isnull().values.any())

True


In [ ]:
print(df.isnull().sum())

score      0
text     106
dtype: int64


In [ ]:
df.loc[df.text.isnull()]

,score,text
7017,1,NaN
7031,1,NaN
14155,1,NaN
14166,1,NaN
14167,1,NaN
...,...,...
24817,1,NaN
32607,1,NaN
32610,1,NaN
32611,1,NaN


In [ ]:
#null값의 샘플을 제거
df = df.dropna(how = 'any')
print(df.isnull().values.any()) #null값이 또 존재 하는지 확인

False


In [ ]:
print(len(df['text']))

38229


In [ ]:
from konlpy.tag import Twitter

twitter = Twitter()

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [ ]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(df['text'], df['score'], test_size = 0.2, random_state=0)

In [ ]:
len(train_x),len(train_y)

(30583, 30583)

In [ ]:
len(test_x), len(test_y)

(7646, 7646)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(tokenizer=twitter.morphs, ngram_range=(1,2), min_df = 3, max_df=0.9)
tfv.fit(train_x)
tfv_train_x = tfv.transform(train_x)
tfv_train_x

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


<30583x32188 sparse matrix of type '<class 'numpy.float64'>'
	with 737966 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.linear_model import LogisticRegression #이진 분류 알고리즘
from sklearn.model_selection import GridSearchCV #하이퍼 파라미터 최적화

clf = LogisticRegression(random_state = 0)
params = {'C':[1,3,5,7,9]}
grid_cv = GridSearchCV(clf, param_grid=params, cv=4, scoring ='accuracy', verbose = 1) #4번의 '모의고사'후 가장 좋은 방법을 찾는 트레이닝
grid_cv.fit(tfv_train_x, train_y)


In [ ]:
grid_cv.best_params_

{'C': 9}

In [ ]:
#100점 만점에 89혹은 10번중 8-9번은 맞는다는뜻
grid_cv.best_score_ 

0.9576889141434576

In [ ]:
#윗 트레이닝은 7048개에 분리된 트레이닝세트로 만들어진 트레이닝으로 89%의 정확도를 보여줌.
#아래는 1762개로 분리된 테스트세트로 정확도를 알려줌.

#테스트세트를 불러줌
tfv_test_x = tfv.transform(test_x)
grid_cv.best_estimator_.score(tfv_test_x, test_y) #거의 비슷한 정확도를 보여줌

0.9593251373267068

**-----------------------------------------------------------------------------Extract keyword--------------------------------------------------------------------**

In [ ]:
#comment에서 아버지 단어 코맨트 scrapping 하기
father = df[df['comment'].str.contains('아버지')]
father

In [ ]:
father.to_csv('father ver2.csv')

**-------------------------------------------------------** 감정 분석 데이터 리스트로 받고서 긍/부 갯수 받기**--------------------------------------------------------**



In [ ]:
import pandas as pd
import numpy as np

ds = pd.read_csv("father ver2.csv")

In [ ]:
ds.tail()

,Unnamed: 0,comment
5044,40213,힘드셨던 우리 아버지 어머니들.그런데 왜 힘들었을까? 왜 힘들었는지에대한 얘긴 왜 ...
5045,40280,"우리 모두의 '아버지'들이 살아온 이야기, 그 자체의 힘. 70대 황정민의 굽은 등..."
5046,40282,"진보, 보수 등 정치논리로 영화를 보지맙시다! 그냥 그 힘겨운 시대에 던져졌으나 끈..."
5047,40283,"솔직히 기자새끼들이 한괴뢰 출신이라 그렇지 재미있는 내용은맞음, 아버지시대의 교훈도..."
5048,40287,좌우막론하고 우리아버지세대들이 대단했다는건 사실이다


In [ ]:
ds['comment'][1:75]

1     20대초반 젊은이로 우리가 겪지못했던 우리한아버지들의삶을 보여주는 영화..그들이 있...
2                            아 눈물범벅. 고생하신 아버지 생각나네요 ㅠㅠ 
3     시사회때 봤는데 단연코 수작임. 2014년 대표영화로 손꼽을만함.근데 방금개봉했는데...
4                 감정 개입  하지말고.. 영화보면서 아버지 생각에 먹먹합니다 .. 
5                               내  아버지가  그랬던 것  처럼 ... 
                            ...                        
70    진짜 꿀잼....내 아버지는 위대하다. 난 나라님이나 하나님보다 내 아버지를 더 존...
71    별10개로는 모자랍니다 할아버지삶이 살아있는역사네요얼마나 울었는지 눈이퉁퉁부었어요 ...
72                  우리 아버지 세대의 고통이 있었기에 오늘의 풍요가 있는 것이죠 
73    &#34;힘들다. 그래도 애들이 안 겪고 내가 겪어서 다행인 것 같다&#34;라는 ...
74    노인네 분장은 삼류, 김윤진사투리는 엉터리로  문제있지만,  시나리오나 연기, CG...
Name: comment, Length: 74, dtype: object

In [ ]:
ds['comment'][0]

'영화내내 가족에게 잘해야겠다 생각이 들었다.우리 아버지는 날위해 모든것을바치신사람이다. 효도해야겠다. '

In [ ]:
a = ds['comment'][0:500].tolist()
a

In [ ]:
 #꼭 리스트로 전달 해야함 여기에 내가 원하는 파일을 리스트로 전달해서 보기가능.
my_review = tfv.transform(a)
grid = grid_cv.best_estimator_.predict(my_review)
grid


In [ ]:
#부정댓글을 찾기
comment_0_rate = numpy.where(grid == 0)
comment_0_rate

(array([ 76, 109, 247, 324]),)

In [ ]:
for i in comment_0_rate:
  print(i)

[ 76 109 247 324]
